In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
test = pd.read_csv('test.csv')
# test.head(10)

In [14]:
train = pd.read_csv('train.csv')
# train.head(10)

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [6]:
train.drop(columns= 'Примечание', axis=1, inplace=True)

In [ ]:
train.dropna()

In [8]:
train.drop(['ЦОС', 'Схема', 'Класс', 'Юридический адрес заявителя', 'Контакты заявителя', 'Контакты изготовителя', 'Органпосертификации', 'Изготовитель', 'Номер бланка', 'Рег. номер cертификата соответствия (СС)', 'Юридический адрес изготовителя'], axis = 1, inplace = True)
#train.head(3)

In [9]:
#train['Дата окончания действия СС'] = pd.to_datetime(train['Дата окончания действия СС'])

In [ ]:
#train['Датавыдачи СС'] = pd.to_datetime(train['Датавыдачи СС'])

In [ ]:
train.head(10)

In [ ]:
'''
time_series = train.groupby('Датавыдачи СС').size()

# Визуализация временного ряда
time_series.plot(figsize=(12, 6), title='Количество продукции по времени')
plt.xlabel('Дата выдачи СС')
plt.ylabel('Количество продукции')
plt.show()
'''

In [15]:
train.set_index('Датавыдачи СС', inplace = True)
#train.head(5)

In [ ]:
train.index = pd.to_datetime(train.index)
train.index[:5]

теперь попробуем подсчитать сколько сертов делаллось за год

In [ ]:
train['Год'] = train.index.year
train['Месяц'] = train.index.month

applicants= train.groupby(['Год', 'Месяц'])['Заявитель'].count().reset_index(name='Общее количество заявителей')

print(applicants)

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='Год', y='Общее количество заявителей', hue='Месяц', data=applicants, marker='o')
plt.title('Общее количество заявителей по годам и месяцам')
plt.xlabel('Год')
plt.ylabel('Общее количество заявителей')
plt.legend(title='Месяц', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

 модель линейная

In [ ]:
frequency = train['Наименование продукции'].value_counts()
print(frequency)

In [20]:
labels, unique = pd.factorize(train['Заявитель'])

# Сохраняем индификаторы
train['Заявитель_id'] = labels

frequency = train['Заявитель_id'].value_counts()
#print(frequency)

In [ ]:
time_series = pd.Series(labels, index=train.index)

print(time_series)

активность конкретных заявителей

In [ ]:
activity_count = train.groupby('Заявитель_id').size()

activity_count_sorted = activity_count.sort_values(ascending=False)

print(activity_count_sorted)

In [ ]:
id_to_applicant = {id: applicant for id, applicant in zip(labels, unique)}

train['Заявитель'] = train['Заявитель_id'].map(id_to_applicant)

applicant_activity = train.groupby('Заявитель').size().sort_values(ascending=False)

top_applicants = applicant_activity.head(10)
print(top_applicants)

проверка на стационарность (делала через визуализацию, не вышло)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(time_series):
    # Выполнение теста Дики-Фуллера:
    result = adfuller(time_series, autolag='AIC')

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    # Интерпретация результатов:
    if result[1] <= 0.05:
        print("Отвергаем нулевую гипотезу (H0), временной ряд стационарен.")
    else:
        print("Не отвергаем нулевую гипотезу (H0), временной ряд нестационарен.")

check_stationarity(time_series)

In [ ]:
time_series_diff = time_series.diff().dropna()

result_diff = adfuller(time_series_diff)
print('ADF Statistic: %f' % result_diff[0])
print('p-value: %f' % result_diff[1])
print('Critical Values:')
for key, value in result_diff[4].items():
    print('\t%s: %.3f' % (key, value))

if result_diff[1] <= 0.05:
    print("Отвергаем нулевую гипотезу (H0), временной ряд стационарен после первого дифференцирования.")
else:
    print("Не отвергаем нулевую гипотезу (H0), временной ряд все еще нестационарен и может потребовать дополнительного дифференцирования.")


In [ ]:
test.set_index('Датавыдачи СС', inplace=True)
test.index = pd.to_datetime(test.index)
test.index[:5]

In [25]:
label_map = {name: label for label, name in enumerate(unique)}
test['Заявитель_id'] = test['Заявитель'].map(label_map)

test['Заявитель_id'].fillna(-1, inplace=True)

test_time_series = pd.Series(test['Заявитель_id'].values, index=test.index)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.figure(figsize=(12, 6))
plot_acf(time_series, lags=25)
plt.title('График автокорреляционной функции (ACF)')
plt.show()

plt.figure(figsize=(12, 6))
plot_pacf(time_series, lags=25)
plt.title('График частичной автокорреляционной функции (PACF)')
plt.show()

Итог, каждый 6 лаг